# Import

In [1]:
%matplotlib widget

In [2]:
import pickle
import os
import pandas as pd
import numpy as np
import json

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import roc_auc_score, roc_curve

In [29]:
sns.set_style("whitegrid")

# Define

In [3]:
def compute_px(ppl, lls):
    lengths = np.array([len(ll) for ll in lls])
    logpx = np.log(ppl) * lengths * -1
    return logpx

def compute_auroc_all(id_msp, id_px, id_ppl, ood_msp, ood_px, ood_ppl, do_print=False):
    score_px = compute_auroc(-id_px, -ood_px)
    score_py = compute_auroc(-id_msp, -ood_msp)
    score_ppl = compute_auroc(id_ppl, ood_ppl)
    if do_print:
        print(f"P(x): {score_px:.3f}")
        print(f"P(y | x): {score_py:.3f}")
        print(f"Perplexity: {score_ppl:.3f}")
    scores = {
        'p_x': score_px,
        'p_y': score_py,
        'ppl': score_ppl
    }
    return scores

In [4]:
def compute_auroc(id_pps, ood_pps, normalize=False, return_curve=False):
    y = np.concatenate((np.ones_like(ood_pps), np.zeros_like(id_pps)))
    scores = np.concatenate((ood_pps, id_pps))
    if normalize:
        scores = (scores - scores.min()) / (scores.max() - scores.min())
    if return_curve:
        return roc_curve(y, scores)
    else:
        return 100*roc_auc_score(y, scores)

def compute_far(id_pps, ood_pps, rate=5):
    incorrect = len(id_pps[id_pps > np.percentile(ood_pps, rate)])
    return 100*incorrect / len(id_pps)

In [5]:
def compute_metric_all(id_msp, id_px, id_ppl, ood_msp, ood_px, ood_ppl, metric='auroc', do_print=False):
    if metric == 'auroc':
        score_px = compute_auroc(-id_px, -ood_px)
        score_py = compute_auroc(-id_msp, -ood_msp)
        score_ppl = compute_auroc(id_ppl, ood_ppl)
    elif metric == 'far':
        score_px = compute_far(-id_px, -ood_px)
        score_py = compute_far(-id_msp, -ood_msp)
        score_ppl = compute_far(id_ppl, ood_ppl)
    else:
        raise Exception('Invalid metric name')

    if do_print:
        print(f"Metric {metric}:")
        print(f"P(x): {score_px:.3f}")
        print(f"P(y | x): {score_py:.3f}")
        print(f"Perplexity: {score_ppl:.3f}\n")

    scores = {
        'p_x': score_px,
        'p_y': score_py,
        'ppl': score_ppl
    }
    return scores

In [6]:
def read_model_out(fname):
    ftype = fname.split('.')[1]
    
    if ftype == 'pkl':
        with open(fname, 'rb') as f:
            return pickle.load(f)
    elif ftype == 'npy':
        return np.load(fname)
    else:
        raise KeyError(f'{ftype} not supported')


# Summarize

## Presettings

In [7]:
verbose = False

In [8]:
repo = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
print(repo)

C:\Users\Willi\Documents\NYU\2020_Fall\nlp\project\ood-detection


In [9]:
output_dir = os.path.join(repo, 'output')
fig_dir = os.path.join(repo, 'figs')

In [10]:
train_sets = ['imdb', 'sst2']
eval_sets = ['imdb', 'sst2', 'snli', 'counterfactual-imdb', 'rte']
methods = ['msp', 'lls', 'pps']

signals = {}
for train_set in train_sets:
    for eval_set in eval_sets:
        signals[(train_set, eval_set)] = {method: None for method in methods}

## Import Signals

In [11]:
method2ftype={
    'msp': 'npy',
    'lls': 'pkl',
    'pps': 'npy',
}

### Subsampling Indices

In [12]:
def get_indices(fname):
    with open(fname, 'r') as f:
        return [int(x) for x in f.readlines()]

In [13]:
roberta_dir = os.path.join(repo, 'roberta')

subsample_indices = {
    data_name: get_indices(os.path.join(roberta_dir, f'{data_name}_indices.txt'))
    for data_name in train_sets
}

### GPT2

In [14]:
best_lr = {
    'imdb': '5e-5',
    'sst2': '5e-5',
}

methods = ['lls', 'pps']
not_readys = []

for (train_set, eval_set), signals_dict in signals.items():
    for method in methods:
        signal_fname = os.path.join(output_dir, 'gpt2', train_set, f'{eval_set}_{best_lr[train_set]}_{method}.{method2ftype[method]}')
        if not os.path.exists(signal_fname):
            not_readys.append((train_set, eval_set, method))
            continue
        
        signal = read_model_out(signal_fname)
        if train_set == eval_set:
            idxs = subsample_indices[train_set]
            signal = [signal[idx] for idx in idxs]
            if method == 'pps':
                signal = np.array(signal)
        
        signals_dict[method] = signal
        
for not_ready in not_readys:
    print(not_ready)

### RoBERTa

In [15]:
methods = ['msp']
not_readys = []

model_type = 'roberta-large'

for (train_set, eval_set), signals_dict in signals.items():
    for method in methods:
        signal_fname = os.path.join(output_dir, 'roberta', train_set, f'{model_type}_{eval_set}_{method}.{method2ftype[method]}')
        if not os.path.exists(signal_fname):
            not_readys.append((train_set, eval_set, method))
            continue

        signals_dict[method] = read_model_out(signal_fname)
        
for not_ready in not_readys:
    print(not_ready)

## Summarize Evals

In [16]:
metrics = ['auroc', 'far']

metric_summary = []

score2plot = {
    'p_x': r'GPT2: $p(x)$',
    'ppl': 'GPT2: PPL',
    'p_y': 'RoBERTa: MSP',
}

metric2plot = {
    'auroc': 'AUROC',
    'far': 'FAR95'
}

dataset2plot = {
    'imdb': 'IMDB',
    'sst2': 'SST-2',
    'snli': 'SNLI',
    'counterfactual-imdb': 'c-IMDB',
    'rte': 'RTE',
}

not_ready = []
for train_set in train_sets:
    for eval_set in eval_sets:
        if train_set == eval_set:
            continue
        
        ood_signal_dict = signals[(train_set, eval_set)]
        id_signal_dict = signals[(train_set, train_set)]
        
        skip=False
        for value in ood_signal_dict.values():
            if isinstance(value, type(None)):
                skip=True
                
        if skip:
            not_ready.append((train_set, eval_set))
            continue
            
        ood_px = compute_px(ood_signal_dict['pps'], ood_signal_dict['lls'])
        id_px = compute_px(id_signal_dict['pps'], id_signal_dict['lls'])
               
        for metric in metrics:
            scores = compute_metric_all(
                id_msp=id_signal_dict['msp'],
                id_px=id_px,
                id_ppl=id_signal_dict['pps'],
                ood_msp=ood_signal_dict['msp'],
                ood_px=ood_px,
                ood_ppl=ood_signal_dict['pps'],
                metric=metric,
                do_print=verbose
            )
            
            for score_key, score_value in scores.items():
                row = {'Method': score2plot[score_key], 'score': score_value}
            
                row['in_domain'] = dataset2plot[train_set]
                row['out_domain'] = dataset2plot[eval_set]
                row['metric'] = metric2plot[metric]

                metric_summary.append(row)        
        
print(pd.DataFrame(metric_summary))

          Method      score in_domain out_domain metric
0   GPT2: $p(x)$   0.394509      IMDB      SST-2  AUROC
1   RoBERTa: MSP  70.281889      IMDB      SST-2  AUROC
2      GPT2: PPL  91.284841      IMDB      SST-2  AUROC
3   GPT2: $p(x)$  99.990000      IMDB      SST-2  FAR95
4   RoBERTa: MSP  80.905000      IMDB      SST-2  FAR95
5      GPT2: PPL  50.785000      IMDB      SST-2  FAR95
6   GPT2: $p(x)$   0.166909      IMDB       SNLI  AUROC
7   RoBERTa: MSP  94.410856      IMDB       SNLI  AUROC
8      GPT2: PPL  71.344013      IMDB       SNLI  AUROC
9   GPT2: $p(x)$  99.980000      IMDB       SNLI  FAR95
10  RoBERTa: MSP  14.000000      IMDB       SNLI  FAR95
11     GPT2: PPL  91.170000      IMDB       SNLI  FAR95
12  GPT2: $p(x)$  41.484174      IMDB     c-IMDB  AUROC
13  RoBERTa: MSP  63.987969      IMDB     c-IMDB  AUROC
14     GPT2: PPL  55.156607      IMDB     c-IMDB  AUROC
15  GPT2: $p(x)$  94.670000      IMDB     c-IMDB  FAR95
16  RoBERTa: MSP  81.240000      IMDB     c-IMDB

# Plot

In [17]:
title_size=24
label_size=18
annot_size=14

In [31]:
def my_plot(
    df=None,
    x='out_domain',
    y='score',
    hue='Method',
    title=None,
    xlabel='',
    ylabel='',
    palette='colorblind',
    ylim=[0,110],
    yticks=np.arange(0,110,20),
    order=None,
    val_offset = 5,
    hue_order=None,
    figsize=(10,5),
    legend_pos='upper left',
    legend_anchor=(1,1),
    legend_on=True,
    ax = None,
):
    if ax is None:
        fig, ax = plt.subplots(figsize=figsize)
        
    splot = sns.barplot(
        data=df, 
        x=x, 
        y=y,
        hue=hue,
        palette=palette,
        hue_order=hue_order,
        order=order,
        ax=ax,
    )
    
    for p in splot.patches:
        splot.annotate(
            f'{p.get_height():.1f}',
            (p.get_x() + p.get_width() / 2, p.get_height() + val_offset),
            ha='center', va='center',
            fontsize=annot_size
        )
    
    if not title is None:
        ax.set_title(title, fontsize=title_size)
        
    ax.set_xlabel(xlabel, fontsize=label_size)
    ax.set_ylabel(ylabel, fontsize=label_size)
    
    ax.legend(loc=legend_pos, bbox_to_anchor=legend_anchor).set_visible(legend_on)
    ax.set_ylim(ylim)
    
    splot.set_yticklabels(yticks, size=annot_size)
    splot.set_xticklabels(order, size=annot_size)
    
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    
    fig.tight_layout()  
    
    if ax is None:
        return fig

In [30]:
summary_df = pd.DataFrame(metric_summary)
xlabel = 'Out-of-Domain Sets'

order_all = ['c-IMDB', 'IMDB', 'SST-2', 'SNLI', 'RTE']
hue_order = ['RoBERTa: MSP',  'GPT2: PPL', r'GPT2: $p(x)$']
palette = {
    'RoBERTa: MSP':'tab:orange',
    'GPT2: PPL':'tab:cyan',
    r'GPT2: $p(x)$':'tab:blue',
}
figsize = (10, 5)

ylim = [0, 115]
legend_pos = 'upper left'
legend_anchor=(0,1.2)

save = False
fig_ftype = 'jpg'

for metric in summary_df['metric'].unique():
    sub_df = summary_df.loc[summary_df['metric'] == metric, :]
    
    for in_domain in sub_df['in_domain'].unique():
        subsub_df = sub_df.loc[sub_df['in_domain'] == in_domain, :]
        
        order=[out_domain for out_domain in order_all if out_domain in subsub_df['out_domain'].unique()]
        
        fig = my_plot(
            df=subsub_df,
#             title=f'{in_domain}',
            xlabel=xlabel,
#             ylabel=metric,
            order=order,
            hue_order=hue_order,
            palette=palette,
            figsize=figsize,
            ylim=ylim,
            legend_pos=legend_pos,
            legend_anchor=legend_anchor,
        )
        
        if save:
            os.makedirs(fig_dir, exist_ok=True)
            fig.savefig(os.path.join(fig_dir, f'{in_domain}_{metric}.{fig_ftype}'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Final Plot

In [ ]:
summary_df = pd.DataFrame(metric_summary)
xlabel = 'Out-of-Domain Sets'

order_all = ['c-IMDB', 'IMDB', 'SST-2', 'SNLI', 'RTE']
hue_order = ['RoBERTa: MSP',  'GPT2: PPL', r'GPT2: $p(x)$']
palette = {
    'RoBERTa: MSP':'tab:orange',
    'GPT2: PPL':'tab:cyan',
    r'GPT2: $p(x)$':'tab:blue',
}
figsize = (20, 10)

ylim = [0, 115]
legend_pos = 'upper left'
legend_anchor=(0,1.2)

save = False
fig_ftype = 'jpg'

fig, ax = plt.subplot(2, 2, figsize=figsize)

for m_idx, metric in enumerate(summary_df['metric'].unique()):
    sub_df = summary_df.loc[summary_df['metric'] == metric, :]
    
    for d_idx, in_domain in enumerate(sub_df['in_domain'].unique()):
        subsub_df = sub_df.loc[sub_df['in_domain'] == in_domain, :]
        
        order=[out_domain for out_domain in order_all if out_domain in subsub_df['out_domain'].unique()]
        
        fig = my_plot(
            df=subsub_df,
#             title=f'{in_domain}',
            xlabel=xlabel,
#             ylabel=metric,
            order=order,
            hue_order=hue_order,
            palette=palette,
            figsize=figsize,
            ylim=ylim,
            legend_pos=legend_pos,
            legend_anchor=legend_anchor,
        )


# Distributions

In [20]:
fig, ax = plt.subplots()
sns.distplot(signals[('imdb', 'imdb')]['pps'])
sns.distplot(signals[('imdb', 'counterfactual-imdb')]['pps'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\Willi\Anaconda3\envs\DL\lib\site-packages\seaborn\distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
C:\Users\Willi\Anaconda3\envs\DL\lib\site-packages\seaborn\distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


In [21]:
compute_auroc(signals[('imdb', 'imdb')]['pps'], signals[('imdb', 'counterfactual-imdb')]['pps'])

55.15660655737704

In [22]:
fig, ax = plt.subplots()
sns.distplot(signals[('imdb', 'imdb')]['msp'])
sns.distplot(signals[('imdb', 'counterfactual-imdb')]['msp'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\Willi\Anaconda3\envs\DL\lib\site-packages\seaborn\distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
C:\Users\Willi\Anaconda3\envs\DL\lib\site-packages\seaborn\distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


In [23]:
fig, ax = plt.subplots()
sns.distplot(compute_px(signals[('imdb', 'imdb')]['pps'], signals[('imdb', 'imdb')]['lls']))
sns.distplot(compute_px(signals[('imdb', 'sst2')]['pps'], signals[('imdb', 'sst2')]['lls']))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\Willi\Anaconda3\envs\DL\lib\site-packages\seaborn\distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
C:\Users\Willi\Anaconda3\envs\DL\lib\site-packages\seaborn\distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


In [24]:
fig, ax = plt.subplots()
sns.distplot(compute_px(signals[('sst2', 'sst2')]['pps'], signals[('sst2', 'sst2')]['lls']))
sns.distplot(compute_px(signals[('sst2', 'imdb')]['pps'], signals[('sst2', 'imdb')]['lls']))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\Willi\Anaconda3\envs\DL\lib\site-packages\seaborn\distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
C:\Users\Willi\Anaconda3\envs\DL\lib\site-packages\seaborn\distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
